In [1]:
# --- core ---
import os
import asyncio
import datetime as dt
from zoneinfo import ZoneInfo
import nest_asyncio
import math
nest_asyncio.apply()   # only needed if running in Jupyter

# --- alpaca trading / data ---
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, StopOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, StockLatestTradeRequest
from alpaca.data.live import StockDataStream
from alpaca.data.enums import DataFeed


# --- optional (only if you want data analysis or plotting) ---
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import yfinance as yf
# import backtrader as bt
# import pytz
#api_key="PKB9SXQSORHZSB1VIXY0"
#secret_key="q4n01bamgGErL7p261A7mu5UvHZOOTO2i6rSsXgJ"



In [ ]:
# latest code should work fine - test on 08/19 

API_KEY    = "PKB9SXQSORHZSB1VIXY0"
API_SECRET = "q4n01bamgGErL7p261A7mu5UvHZOOTO2i6rSsXgJ"

trading = TradingClient(API_KEY, API_SECRET, paper=True)
client = StockHistoricalDataClient(API_KEY, API_SECRET)

symbol = "PRFX"

# look back a week to be safe
start = dt.datetime.utcnow() - dt.timedelta(days=7)

req = StockBarsRequest(
    symbol_or_symbols=symbol,
    timeframe=TimeFrame.Day,
    start=start,
)

bars = client.get_stock_bars(req).df

# If multiple days, take the second-to-last bar = yesterday's close
bars = bars.xs(symbol, level="symbol") if "symbol" in bars.index.names else bars
bars = bars.sort_index()

if len(bars) >= 2:
    prior_close = float(bars["close"].iloc[-2])
    prior_ts    = bars.index[-2] 
else:
    prior_close = float(bars["close"].iloc[-1])
    prior_ts    = bars.index[-1] 

print(f"Yesterday's close for {symbol}: {prior_close} @ {prior_ts}")
#print(bars)

state = {
    "activated": False,
    "shares": 0,         # negative = short
    "proceeds": 0.0,     # sum(|qty| * price) for shorts
    "mwap": 0.0,         # proceeds / |shares|
}

activated = False
stream = StockDataStream(API_KEY, API_SECRET, feed=DataFeed.IEX)  # or SIP if you have subscription


async def on_trade(trade):
    print("TICK!")
    if trade.symbol != symbol:
        return

    price = float(trade.price)
    ts    = trade.timestamp
    print(f"Live tick: {trade.symbol} {price} at {ts}")

    # 1) Trigger
    if (not state["activated"]) and (price / prior_close) >= 1.5:  # 5% above prior close
        state["activated"] = True

        init_qty = 100                     # initial short (shares)
        # place initial short (SELL init_qty)
        order = trading.submit_order(MarketOrderRequest(
            symbol=symbol,
            qty=init_qty,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.DAY
        ))

        #market_order = trading.submit_order(order_data=order)


        # update short state: shares negative
        state["shares"]   += init_qty
        state["proceeds"] += init_qty * price
        q = abs(state["shares"])
        state["mwap"]     = state["proceeds"] / q
        print(f"[ENTER] long {init_qty} @ ~{price:.4f} | pos={state['shares']} | MWAP={state['mwap']:.4f}")

        return  # done on this tick

    # 2) Ladder adds (only after activated and already short)
    if state["activated"] and state["shares"] > 0:
        target = 0.9 * price
        M = state["mwap"]
        if M < 3.0:  # don't ladder if MWAP is too low
            # dq = q*(T - M) / (P - T)  ; add_shares = int(dq) >= 1
            q = abs(state["shares"])
            if price > target:  # guard divide by zero
                dq = (q * (target - M)) / (price - target)
                add = max(1, int(math.floor(dq)))  # integer shares to ADD
                # submit ONLY the delta
                order = trading.submit_order(MarketOrderRequest(
                    symbol=symbol,
                    qty=add,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                ))
                # update state
                state["shares"]   += add
                state["proceeds"] += add * price
                state["mwap"]      = state["proceeds"] / abs(state["shares"])
                print(f"[LADDER] add {add} @ ~{price:.4f} | pos={state['shares']} | MWAP={state['mwap']:.4f}")

    if state["activated"] and state["shares"] > 0:
        # 3) Stop loss
        if price <= 0.9 * state["mwap"]:
            order = trading.submit_order(MarketOrderRequest(
                symbol=symbol,
                qty=abs(state["shares"]),
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
            ))
            print(f"[EXIT] stop loss @ ~{price:.4f} | pos={state['shares']} | MWAP={state['mwap']:.4f}")    

stream.subscribe_trades(on_trade, symbol)

# run
import asyncio
asyncio.run(stream._run_forever())

C:\Users\YOGA\AppData\Local\Temp\ipykernel_16712\2289369349.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start = dt.datetime.utcnow() - dt.timedelta(days=7)


Yesterday's close for PRFX: 1.34 @ 2025-08-18 04:00:00+00:00
TICK!
Live tick: PRFX 2.21 at 2025-08-19 17:11:02.360357+00:00
[ENTER] long 100 @ ~2.2100 | pos=100 | MWAP=2.2100
TICK!
Live tick: PRFX 2.23 at 2025-08-19 17:11:12.400601+00:00
[LADDER] add 1 @ ~2.2300 | pos=101 | MWAP=2.2102
TICK!
Live tick: PRFX 2.24 at 2025-08-19 17:11:17.561301+00:00
[LADDER] add 1 @ ~2.2400 | pos=102 | MWAP=2.2105
TICK!
Live tick: PRFX 2.21 at 2025-08-19 17:12:11.568891+00:00
[LADDER] add 1 @ ~2.2100 | pos=103 | MWAP=2.2105
TICK!
Live tick: PRFX 2.24 at 2025-08-19 17:12:50.860479+00:00
[LADDER] add 1 @ ~2.2400 | pos=104 | MWAP=2.2108
TICK!
Live tick: PRFX 2.25 at 2025-08-19 17:13:01.824106+00:00
[LADDER] add 1 @ ~2.2500 | pos=105 | MWAP=2.2111
TICK!
Live tick: PRFX 2.23 at 2025-08-19 17:14:22.913400+00:00
[LADDER] add 1 @ ~2.2300 | pos=106 | MWAP=2.2113
TICK!
Live tick: PRFX 2.29 at 2025-08-19 17:15:19.703896+00:00
[LADDER] add 1 @ ~2.2900 | pos=107 | MWAP=2.2121
TICK!
Live tick: PRFX 2.25 at 2025-08-19 

data websocket error, restarting connection: no close frame received or sent


TICK!
Live tick: PRFX 2.3 at 2025-08-19 18:33:45.002273+00:00
[LADDER] add 1 @ ~2.3000 | pos=172 | MWAP=2.2176
TICK!
Live tick: PRFX 2.29 at 2025-08-19 18:33:46.186589+00:00
[LADDER] add 1 @ ~2.2900 | pos=173 | MWAP=2.2181
TICK!
Live tick: PRFX 2.3 at 2025-08-19 18:35:22.749892+00:00
[LADDER] add 1 @ ~2.3000 | pos=174 | MWAP=2.2185
TICK!
Live tick: PRFX 2.28 at 2025-08-19 18:36:24.765658+00:00
[LADDER] add 1 @ ~2.2800 | pos=175 | MWAP=2.2189
TICK!
Live tick: PRFX 2.27 at 2025-08-19 18:36:28.520581+00:00
[LADDER] add 1 @ ~2.2700 | pos=176 | MWAP=2.2192
TICK!
Live tick: PRFX 2.3 at 2025-08-19 18:37:45.972481+00:00
[LADDER] add 1 @ ~2.3000 | pos=177 | MWAP=2.2196
TICK!
Live tick: PRFX 2.3 at 2025-08-19 18:38:09.582986+00:00
[LADDER] add 1 @ ~2.3000 | pos=178 | MWAP=2.2201
TICK!
Live tick: PRFX 2.3 at 2025-08-19 18:38:11.113575+00:00
[LADDER] add 1 @ ~2.3000 | pos=179 | MWAP=2.2205
TICK!
Live tick: PRFX 2.3 at 2025-08-19 18:38:11.114142+00:00
[LADDER] add 1 @ ~2.3000 | pos=180 | MWAP=2.221

In [ ]:
await stream.stop()
await stream_task
print("Stream stopped.")